## combining individual coverage data into a mean-normalized coverage

## Plan:

+ for the PanelofNormals, create a coverage file for each PON bam and each chromosome
+ Pon bams should optimally not contain CNV or there should be a lot of them to reduce the std
+ put all the files into a big matrix and normalize coverages and produce an average coverage (+ std) for the exonic space
+ compare the tumor samples against that PONcoverage to get differences in CNV

### have to look at the Kenichi-CNV-graphs to identify the samples from FDAH1 without CNVs in the normals
+ [1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [3]:
# set the paths
home = '/Users/mahtin'
home = '/Users/martinscience'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
tooldata = f"{home}/Dropbox/Icke/Work/somVar/tooldata"
shell_path = "../shell"
static_path = f"{home}/Dropbox/Icke/Work/static"
bed_path = f"{static_path}/bed_files/SureSelect/hg38"

cnvdata = f"{tooldata}/myCNVdata/"
bedCov_path = f"{cnvdata}/bedCov"
bedCov_path = f"{home}/mount/scratch/develop/PONcoverage/bedCov"
output_path = f"{cnvdata}/output"

In [4]:
bedCov_path

'/Users/martinscience/mount/scratch/develop/PONcoverage/bedCov'

In [5]:
test = pd.read_csv(f'{bedCov_path}/009_B.chr5.bedCov', sep='\t', compression="gzip")
test[:13]

,Chr,Pos,Coverage,ExonPos
0,chr5,140300,184.30,33
1,chr5,140350,242.62,83
2,chr5,140400,288.67,133
3,chr5,140450,347.07,183
4,chr5,140500,406.70,233
5,chr5,140550,480.91,283
6,chr5,140600,544.79,333
7,chr5,140650,523.75,383
8,chr5,140700,384.47,433
9,chr5,140750,187.23,483


### get all the normal samples (without CNV) into df for normalization and averaging

In [7]:
# try with pd merge
import os
samples = [1, 2, 4, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16]
cov_df = pd.DataFrame(columns=['ExonPos', 'Pos'])
for sample in [f"{str(s).zfill(3)}_B" for s in samples]:
    file = os.path.join(bedCov_path, f"{sample}.chr7.bedCov")
    print(f"Reading {sample} from {file}.")
    df = pd.read_csv(file, sep='\t', compression='gzip').loc[:,['Pos', 'ExonPos', 'Coverage']].rename(columns={'Coverage':sample})
    cov_df = cov_df.merge(df, on=['ExonPos', 'Pos'], how='outer')

cov_df = cov_df.fillna(0).sort_values('ExonPos').set_index(['Pos', 'ExonPos'])
cov_df
cov_df.to_csv(f"{output_path}/PON_coverage.csv", sep='\t', index=False)

Reading 001_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/001_B.chr7.bedCov.
Reading 002_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/002_B.chr7.bedCov.
Reading 004_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/004_B.chr7.bedCov.
Reading 006_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/006_B.chr7.bedCov.
Reading 008_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/008_B.chr7.bedCov.
Reading 009_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/009_B.chr7.bedCov.
Reading 010_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/010_B.chr7.bedCov.
Reading 011_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/011_B.chr7.bedCov.
Reading 012_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/012_B.chr7.bedCov.
Reading 013_B from /Users/martinscience/mount/scratch/develop/PONcoverage/bedCov/013_B.chr7.bedCov.


### make a stacked "tidy version" of the coverage df for vizualisation in tidyverse

In [27]:
def tidy_df(df):
    return df.reset_index().drop(columns='Pos').set_index('ExonPos').stack().reset_index().rename(columns={'level_1':'sample', 0:'Coverage'})

tidy_df(cov_df).to_csv(f"{output_path}/PON_coverage_tidy.csv", sep='\t', index=False)

### normalize the coverage to an arbitrary coverage of 250 

In [29]:
# normalize sample coverages to an average coverage of 250
norm_cov = cov_df / cov_df.mean() * 250
norm_cov

tidy_df(norm_cov).to_csv(f"{output_path}/PON_coverage_normtidy.csv", sep='\t', index=False)
# norm_cov.to_csv(f"{bedCov}/all_norm.bedCov", sep='\t')

# normtidy = norm_cov.stack().reset_index().set_index('#').rename(columns={'level_1':'sample', 0:'Coverage'})
# normtidy.to_csv(f"{bedCov}/all_norm_tidy.bedCov", sep='\t')
# normtidy

### avg_cov has the coverage of the "panel of normals"

In [32]:
chrom = "chr7"
avg_cov = pd.DataFrame(index=norm_cov.index)
avg_cov['meanCov'] = norm_cov.mean(axis=1)
avg_cov['medianCov'] = norm_cov.median(axis=1)
avg_cov['std'] = norm_cov.std(axis=1)

avg_cov = avg_cov.reset_index()
avg_cov['Chr'] = chrom
avg_cov = avg_cov.reset_index().loc[:, ['Chr', 'Pos', 'ExonPos', 'meanCov', 'std']]
avg_cov
avg_cov.to_csv(f"{output_path}/PON_coverage.chr7.csv", sep='\t', index=False)